In [43]:
import utils
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

## Data

In [44]:
url = 'https://drive.google.com/file/d/1dTEHbfhnhQxJ_9cKHJCoWaabhofvgH9s/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

train_df = pd.read_csv(url)

url = 'https://drive.google.com/file/d/1-SB_3mNX6ICzoaavq-5DIYNrcL6YPbjc/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

valid_df = pd.read_csv(url)

url = 'https://drive.google.com/file/d/1X3rYSVNuRWBe3XUHMkRjMqgpfKpl5L34/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

test_df = pd.read_csv(url)

train_df['reviews'] = train_df['reviews'].astype(str)
valid_df['reviews'] = valid_df['reviews'].astype(str)
test_df['reviews'] = test_df['reviews'].astype(str)

url = 'https://drive.google.com/file/d/1yF86bZBUDgKPGjmd-daJnBKgxaehByff/view?usp=drive_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
data_df = pd.read_csv(url)

data_df['reviews'] = data_df['reviews'].astype(str)

data_df.columns = train_df.columns

train_data_df, valid_data_df = train_test_split(data_df, train_size=4000, random_state=42)

train_df = pd.concat([train_df, train_data_df], ignore_index=True)
valid_df = pd.concat([valid_df, valid_data_df], ignore_index=True)

url = 'https://drive.google.com/file/d/14MndabbQdo8LCzm40ReGZkGAo0G1wb4r/view?usp=sharing'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
data_df = pd.read_csv(url)
data_df
data_df['reviews'] = data_df['reviews'].astype(str)
data_df.columns = train_df.columns
data_df.info()

train_df = pd.concat([train_df, data_df], ignore_index=True)

print(f"Length of training data: {len(train_df)}")
print(f"Length of validation data: {len(valid_df)}")
print(f"Length of test data: {len(test_df)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654 entries, 0 to 1653
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   reviews   1654 non-null   object
 1   negative  1654 non-null   bool  
 2   neutral   1654 non-null   bool  
 3   positive  1654 non-null   bool  
dtypes: bool(3), object(1)
memory usage: 17.9+ KB
Length of training data: 16654
Length of validation data: 1595
Length of test data: 500


In [45]:
data = {
    'reviews': [
        'siaap',
        'siaapp',
        'siaaap',
        'siaaappp',
        'siaaaap',
        'siaaaap',
        'siaaaap bang',
    ],
    'negative': [False] * 7,
    'neutral': [True] * 7,
    'positive': [False] * 7,
}

df_data = pd.DataFrame(data)

train_df =  pd.concat([train_df, df_data], axis=0).reset_index(drop=True)

print(len(train_df))

16661


## Model

In [46]:
model = utils.load_nlp_model('model/bert_attention_v7_t.h5')
tokenizer = utils.load_tokenizer('tokenizer')

In [48]:
model.compile(optimizer=tf.keras.optimizers.Nadam(5e-2), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 128, 768)  │ 24,516,864 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 128, 256)  │    918,528 │ embedding_3[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_2         │ (None, 128, 256)  │          0 │ bidirectional_3[… │
│ (Attention)         │                   │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ attention_2[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     32,896 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │        387 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25,468,675 (97.16 MB)

 Trainable params: 951,811 (3.63 MB)

 Non-trainable params: 24,516,864 (93.52 MB)

In [49]:
train_reviews = train_df['reviews']
valid_reviews = valid_df['reviews']
test_reviews = test_df['reviews']

train_pre_reviews = [utils.preprocess_text(text) for text in train_reviews]
valid_pre_reviews = [utils.preprocess_text(text) for text in valid_reviews]
test_pre_reviews = [utils.preprocess_text(text) for text in test_reviews]

In [51]:
y_train = train_df.drop(columns='reviews')
y_valid = valid_df.drop(columns='reviews')
y_test = test_df.drop(columns='reviews')

In [52]:
X_train_tokenized = utils.tokenize_batch(train_pre_reviews, tokenizer=tokenizer)
X_valid_tokenized = utils.tokenize_batch(valid_pre_reviews, tokenizer=tokenizer)
X_test_tokenized = utils.tokenize_batch(test_pre_reviews, tokenizer=tokenizer)

In [53]:
X_train_tokenized

<tf.Tensor: shape=(16661, 128), dtype=int32, numpy=
array([[   3, 8389, 4822, ...,    0,    0,    0],
       [   3, 9153, 4515, ...,    0,    0,    0],
       [   3, 3197, 6783, ...,    0,    0,    0],
       ...,
       [   3, 7083, 7823, ...,    0,    0,    0],
       [   3, 7083, 7823, ...,    0,    0,    0],
       [   3, 7083, 7823, ...,    0,    0,    0]], dtype=int32)>

In [54]:
model.fit   (X_train_tokenized, y_train,
            epochs=3,
            validation_data=(X_valid_tokenized, y_valid),
            )

Epoch 1/3
521/521 ━━━━━━━━━━━━━━━━━━━━ 82s 149ms/step - accuracy: 0.6218 - loss: 1.4488 - val_accuracy: 0.7605 - val_loss: 0.5758
Epoch 2/3
521/521 ━━━━━━━━━━━━━━━━━━━━ 82s 158ms/step - accuracy: 0.7941 - loss: 0.5493 - val_accuracy: 0.8031 - val_loss: 0.4924
Epoch 3/3
521/521 ━━━━━━━━━━━━━━━━━━━━ 95s 183ms/step - accuracy: 0.8130 - loss: 0.5126 - val_accuracy: 0.7918 - val_loss: 0.5282


In [41]:
model.save('model/bert_attention_siap_v2.h5')